# Scaling for Production

## Your first application

In [1]:
import numpy as np

num_items = 100  # results in 10100 binary variables

weights = np.random.default_rng(42).random(num_items)

In [2]:
import typing

import dimod


def bin_packing(weights: typing.Sequence[float]) -> dimod.ConstrainedQuadraticModel:
    """Generate a bin packing problem as a constrained quadratic model."""

    n = len(weights)
    
    # y_j indicates that bin j is used
    y = [dimod.Binary(f'y_{j}') for j in range(n)]
    
    # x_i,j indicates that item i is put in bin j
    x = [[dimod.Binary(f'x_{i},{j}') for j in range(n)] for i in range(n)]
    
    cqm = dimod.ConstrainedQuadraticModel()
    
    # we wish to minimize the number of bins used
    cqm.set_objective(sum(y))
    
    # each item can only go in one bin
    for i in range(n):
        cqm.add_constraint(sum(x[i]) == 1, label=f'item_placing_{i}')
        
    # each bin has a capacity that must be respected
    for j in range(n):
        cqm.add_constraint(sum(weights[i] * x[i][j] for i in range(n)) - y[j] <= 0,
                           label=f'capacity_bin_{j}')
        
    return cqm

In [3]:
%timeit bin_packing(weights)

365 ms ± 4.81 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


## Use quicksum

In [4]:
import typing

import dimod


def bin_packing(weights: typing.Sequence[float]) -> dimod.ConstrainedQuadraticModel:
    """Generate a bin packing problem as a constrained quadratic model."""

    n = len(weights)
    
    # y_j indicates that bin j is used
    y = [dimod.Binary(f'y_{j}') for j in range(n)]
    
    # x_i,j indicates that item i is put in bin j
    x = [[dimod.Binary(f'x_{i},{j}') for j in range(n)] for i in range(n)]
    
    cqm = dimod.ConstrainedQuadraticModel()
    
    # we wish to minimize the number of bins used
    cqm.set_objective(dimod.quicksum(y))
    
    # each item can only go in one bin
    for i in range(n):
        cqm.add_constraint(dimod.quicksum(x[i]) == 1, label=f'item_placing_{i}')
        
    # each bin has a capacity that must be respected
    for j in range(n):
        cqm.add_constraint(dimod.quicksum(weights[i] * x[i][j] for i in range(n)) - y[j] <= 0,
                           label=f'capacity_bin_{j}')
        
    return cqm

In [5]:
%timeit bin_packing(weights)

286 ms ± 4.18 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


## Construct the models individually

In [6]:
def make_bqm_symbolic(num_variables: int) -> dimod.BinaryQuadraticModel:
    return dimod.quicksum(2*dimod.Binary(v) for v in range(num_variables))

def make_bqm_labels(num_variables: int) -> dimod.BinaryQuadraticModel:
    bqm = dimod.BinaryQuadraticModel('BINARY')
    bqm.add_linear_from((v, 2) for v in range(num_variables))
    return bqm

In [7]:
%timeit make_bqm_symbolic(1000)
%timeit make_bqm_labels(1000)

13.3 ms ± 197 µs per loop (mean ± std. dev. of 7 runs, 100 loops each)
191 µs ± 1.24 µs per loop (mean ± std. dev. of 7 runs, 10000 loops each)


In [8]:
import typing

import dimod


def bin_packing(weights: typing.Sequence[float]) -> dimod.ConstrainedQuadraticModel:
    """Generate a bin packing problem as a constrained quadratic model."""

    n = len(weights)
    
    # y_j indicates that bin j is used
    y_labels = [f'y_{j}' for j in range(n)]
    
    # x_i,j indicates that item i is put in bin j
    x_labels = [[f'x_{i},{j}' for j in range(n)] for i in range(n)]
    
    cqm = dimod.ConstrainedQuadraticModel()
    
    # we wish to minimize the number of bins used
    objective = dimod.QuadraticModel()
    objective.add_linear_from(((v, 1) for v in y_labels), default_vartype='BINARY')
    cqm.set_objective(objective)
    
    # each item can only go in one bin
    for i in range(n):
        lhs = dimod.QuadraticModel()
        lhs.add_linear_from(((v, 1) for v in x_labels[i]), default_vartype='BINARY')
        cqm.add_constraint_from_model(lhs, rhs=1, sense='==', label=f'item_placing_{i}')
        
    # each bin has a capacity that must be respected
    for j in range(n):
        lhs = dimod.QuadraticModel()
        lhs.add_linear_from(((x_labels[i][j], weights[i]) for i in range(n)), default_vartype='BINARY')
        lhs.add_linear(y_labels[j], -1, default_vartype='BINARY')
        cqm.add_constraint_from_model(lhs, rhs=0, sense='<=', label=f'capacity_bin_{j}')
        
    return cqm

In [9]:
%timeit bin_packing(weights)

98.7 ms ± 1.78 ms per loop (mean ± std. dev. of 7 runs, 10 loops each)


## Don't copy constraints

In [10]:
import typing

import dimod


def bin_packing(weights: typing.Sequence[float]) -> dimod.ConstrainedQuadraticModel:
    """Generate a bin packing problem as a constrained quadratic model."""

    n = len(weights)
    
    # y_j indicates that bin j is used
    y_labels = [f'y_{j}' for j in range(n)]
    
    # x_i,j indicates that item i is put in bin j
    x_labels = [[f'x_{i},{j}' for j in range(n)] for i in range(n)]
    
    cqm = dimod.ConstrainedQuadraticModel()
    
    # we wish to minimize the number of bins used
    objective = dimod.QuadraticModel()
    objective.add_linear_from(((v, 1) for v in y_labels), default_vartype='BINARY')
    cqm.set_objective(objective)
    
    # each item can only go in one bin
    for i in range(n):
        lhs = dimod.QuadraticModel()
        lhs.add_linear_from(((v, 1) for v in x_labels[i]), default_vartype='BINARY')
        cqm.add_constraint_from_model(lhs, rhs=1, sense='==', label=f'item_placing_{i}', copy=False)
        
    # each bin has a capacity that must be respected
    for j in range(n):
        lhs = dimod.QuadraticModel()
        lhs.add_linear_from(((x_labels[i][j], weights[i]) for i in range(n)), default_vartype='BINARY')
        lhs.add_linear(y_labels[j], -1, default_vartype='BINARY')
        cqm.add_constraint_from_model(lhs, rhs=0, sense='<=', label=f'capacity_bin_{j}', copy=False)
        
    return cqm

In [11]:
%timeit bin_packing(weights)

66.5 ms ± 319 µs per loop (mean ± std. dev. of 7 runs, 10 loops each)
